## 2. Obtenemos la siguientes medidas:

### Ejemplo: obtener la frecuencia de cada término por especialidad ( nº de ficheros en el que aparece un término en una especialidad)

In [2]:
import os
import pickle

In [3]:
def deserialize_object(path):
    pickle_in = open(path,"rb")
    obj = pickle.load(pickle_in)
    pickle_in.close()
    print("Cargado el objeto", path.split("/")[- 1])
    return obj

In [4]:
path_dic = 'dic_specialties.pkl'
dic_spe = deserialize_object(path_dic)

Cargado el objeto dic_specialties.pkl


In [14]:
def read_diccionary_specialties(dic_spe, path_out_file):
    cont = 0
    for specialty, dic in dic_spe.items():
        
        if cont == 0:
            print("Especialidad:", specialty)

            list_doc_specialty = dic['docs']
            dic_terms = dic['terms']
            
            print("%-53s %-3s %-3s" % ("Término",  "Nº apar", "Nº docs"))
            for term, list_doc_term in dic_terms.items():
                
                print("%-55s -> %-3s %-3s" % (term,  len(list_doc_term), len(set(list_doc_term))))
                #print("\tTérmino:",term, ' - Nº de apariciones:', len(list_doc_term), ' - Nº de docs donde aparece:', len(set(list_doc_term)))
            cont += 1
        
read_diccionary_specialties(dic_spe, path_out_file)    

Especialidad: H02.403.429.730_rheumatology
Término                                               Nº apar Nº docs
('la', 'xerostomía', 'en')                              -> 1   1  
(':', 'rodilla')                                        -> 1   1  
('la', 'electrocardiografiĺa', 'seriada')               -> 1   1  
('repetabs', "''", 'en')                                -> 1   1  
(None, 'papiloflebitis')                                -> 1   1  
('el', 'mecanismo', 'autoinmune')                       -> 1   1  
('educación', 'sanitaria')                              -> 2   2  
('1994-1999', 'en')                                     -> 1   1  
(None, 'respuesta', 'a')                                -> 6   6  
niños                                                   -> 23  23 
('homologías', 'y')                                     -> 1   1  
(':', 'una', 'nomenclatura')                            -> 1   1  
('específico', 'para')                                  -> 2   2  
('séricos', 'en'

('del', 'suero', 'en')                                  -> 1   1  
('cogan', 'y', 'espondiloartritis')                     -> 1   1  
('prednisona', 'en', 'reumatología')                    -> 1   1  
('en', 'pacientes', 'reumáticos')                       -> 2   2  
('difusa', 'asociada', 'a')                             -> 1   1  
('de', 'polimorfonuclear')                              -> 1   1  
('comparativo', 'con', 'otras')                         -> 1   1  
laminar                                                 -> 1   1  
('algunas', 'consideraciones')                          -> 4   4  
('la', 'retino-fluoresceíno-grafía')                    -> 1   1  
('en', 'gota')                                          -> 1   1  
('el', 'diagnóstico', 'del')                            -> 1   1  
(None, None, 'trastorno')                               -> 2   2  
('osteoartritis', 'temporomandibular', '.')             -> 1   1  
('recientes', 'adquisiciones', 'en')                    -> 2  

('un', 'paciente')                                      -> 14  14 
(None, 'titulación')                                    -> 1   1  
('frecuencia', 'del')                                   -> 1   1  
(None, 'algunas')                                       -> 7   7  
('algunos', 'aspectos', 'clínicos')                     -> 1   1  
('factor', 'necrotizante')                              -> 1   1  
('nos', 'reumáticos', '.')                              -> 1   1  
('25', '(')                                             -> 1   1  
(None, 'disminución')                                   -> 3   3  
('rituximab', 'en', 'paciente')                         -> 1   1  
th1/th2                                                 -> 1   1  
('consideraciones', 'diagnósticas', 'y')                -> 1   1  
concepto                                                -> 10  10 
('en', '21', 'familias')                                -> 1   1  
reportadas                                              -> 1  

('de', 'tres', 'situaciones')                           -> 1   1  
('cinética', 'de')                                      -> 1   1  
(None, None, 'características')                         -> 18  18 
('de', 'salud')                                         -> 13  13 
osteomielitis                                           -> 7   7  
('en', 'el', 'méxico')                                  -> 1   1  
('y', 'severidad', 'de')                                -> 1   1  
('sarcoidosis', '.', None)                              -> 2   2  
('aislamiento', 'del', 'microrganismo')                 -> 1   1  
('estudiados', 'por', 'electroforesis')                 -> 1   1  
seudovasculíticos                                       -> 1   1  
('unidad', 'de', 'prevención')                          -> 1   1  
('dolor', 'en', 'mujeres')                              -> 1   1  
('meningococo', '.')                                    -> 1   1  
(None, 'farmacogenética', 'del')                        -> 1  

('en', 'una', 'clínica')                                -> 1   1  
('osteoarticular', ':')                                 -> 1   1  
('linfocito', 'b', ':')                                 -> 1   1  
infantiles                                              -> 6   6  
(None, 'sindrome', 'del')                               -> 1   1  
(None, 'ultimos', 'avances')                            -> 1   1  
('artritis', 'reumatoide', 'en')                        -> 25  25 
('de', 'la', 'lucha')                                   -> 1   1  
('real', 'como')                                        -> 1   1  
('nuestras', 'experiencias', 'inmunológicas')           -> 1   1  
colágeno-pvp                                            -> 1   1  
('prolongada', 'en')                                    -> 1   1  
('incapacidades', 'de')                                 -> 1   1  
('tratada', 'con', 'corticosteroides')                  -> 1   1  
osificante                                              -> 1  

...

('epidermólisis', 'ampollosa', 'congénita')             -> 1   1  
bajas                                                   -> 3   3  
('osteoporótica', ':', 'valoración')                    -> 1   1  
autoagresión                                            -> 1   1  
('infecciones', 'graves', 'en')                         -> 1   1  
('quién', 'financia')                                   -> 1   1  
eficaciay                                               -> 1   1  
('control', ',')                                        -> 1   1  
('terapéutica', 'hormonal')                             -> 2   2  
('tratamiento', 'del', 'síndrome')                      -> 1   1  
('celular', 'de')                                       -> 1   1  
('a', 'la', 'fibromialgia')                             -> 1   1  
('bonaerense', '.', None)                               -> 1   1  
('43', 'casos', '.')                                    -> 1   1  
('scheletriche', 'tra', 'rischi')                       -> 1  

('descriptivo', 'en', 'el')                             -> 1   1  
dicha                                                   -> 1   1  
('y', 'cardiopatía', 'reumatica')                       -> 1   1  
('crónica', 'primaria')                                 -> 2   2  
('de', 'biomarcadores')                                 -> 1   1  
('enfermedad', 'rara', 'en')                            -> 1   1  
('gota', ';', 'el')                                     -> 1   1  
(None, 'condroprotección')                              -> 1   1  
('of', 'secondary')                                     -> 1   1  
randomizado                                             -> 1   1  
('salud', 'osteoarticular')                             -> 1   1  
('de', 'rodilla', 'candidatos')                         -> 1   1  
('en', 'sangre', 'periférica')                          -> 2   2  
lipídico                                                -> 3   3  
('aspectos', 'clínicos', 'y')                           -> 2  

('ehlers-danlos', '.')                                  -> 10  10 
('examenes', 'isotópicos')                              -> 1   1  
('cadera', 'no')                                        -> 1   1  
(None, 'medida', 'y')                                   -> 1   1  
('e', 'ibuprofen', 'en')                                -> 1   1  
('broncopulmonar', '.', None)                           -> 1   1  
('incidencia', 'y', 'características')                  -> 2   2  
utilización                                             -> 17  17 
('en', 'amiloidosis')                                   -> 1   1  
('y', 'crecimiento')                                    -> 1   1  
('infrecuente', 'de', 'suboclusión')                    -> 1   1  
('no', 'secretor', '.')                                 -> 1   1  
('clänicas', 'del', 'reumatismo')                       -> 1   1  
cronica                                                 -> 1   1  
('reumatica', '.')                                      -> 11 

('en', 'el', 'trasplantado')                            -> 1   1  
('enfermedades', 'biliares', 'y')                       -> 1   1  
('hallazgo', 'específico', 'y')                         -> 1   1  
('de', 'artritis', 'reactiva')                          -> 1   1  
('eosinofilia', 'secundaria')                           -> 1   1  
abordar                                                 -> 1   1  
('para', 'la', 'prescripción')                          -> 1   1  
('clínico', 'y', 'límites')                             -> 1   1  
blys                                                    -> 1   1  
('b', "''", 'en')                                       -> 1   1  
('aspectos', 'actuales', 'de')                          -> 1   1  
('valor', 'clinico', 'de')                              -> 1   1  
administrados                                           -> 2   2  
('de', 'c1', 'inhibidor')                               -> 1   1  
('epicondilitis', '.')                                  -> 1  

('parálisis', 'faciales', 'periféricas')                -> 1   1  
('profunda', ',', 'no')                                 -> 1   1  
('pacientes', 'con', 'distrofia')                       -> 1   1  
('temporal', ':')                                       -> 2   2  
('asociadas', 'a', 'la')                                -> 4   4  
('en', 'medicina')                                      -> 1   1  
('sepsis', 'grave')                                     -> 1   1  
acuatico                                                -> 1   1  
('de', 'rosenberg', 'en')                               -> 1   1  
('a', 'comunicacion', 'interauricular')                 -> 1   1  
('lupus', 'eritematoso', 'cutáneo')                     -> 1   1  
('ácido', 'tiaprofénico', '.')                          -> 1   1  
('superficiales', 'en', 'la')                           -> 1   1  
('completo', 'en', 'pacientes')                         -> 1   1  
('de', 'mucosas', 'en')                                 -> 1  

### XXXX. Medidas antiguas: (conservar por si acaso)


In [7]:
# Et: nº de especialidades que contienen el ngram
def obtain_et(term, dic_spe):
    et = 0

    for specialty, dic in dic_spe.items():
        
        # obtenemos el diccionario de términos
        dic_term = dic['terms']
        
         # si el término se encuentra en el diccionario sumamos 1
        if term in dic_term:
            et += 1
            
    return et      

In [8]:
#Número de documentos en las OTRAS especialidades que tienen el n-gram
def obtain_ndpe(term, dic_spe, specialty_actual):
    
    ndpe = 0

    for specialty, dic in dic_spe.items():
        
        # si la especialidad no es la misma
        if not specialty == specialty_actual:
            
            # obtenemos el diccionario de términos
            dic_term = dic['terms']
            
            # si el término se encuentra en el diccionario
            if term in dic_term:
                
                # obtenemos los documentos en los que aparece
                doc_term = list(set(dic_term[term]))
                
                # contamos el nº de documentos y sumamos
                ndpe += len(doc_term)
                
    return ndpe
    

In [9]:
#Nº de documentos en la especialidad que NO tienen el n-gram
def obtain_ndep(list_doc_term, list_doc_total_spe):
    return len(set(set(list_doc_total_spe)-set(list_doc_term)))

In [10]:
def read_diccionary_specialties(dic_spe):
    
    new_dic_metrics = {}
    
    for specialty, dic in dic_spe.items():
        
        print("Especialidad:", specialty)
        
        dic_term = dic['terms']
        list_doc_specialty = dic['docs']
        
        new_dic_term = {}
        
        for term, list_doc_term in dic_term.items():
            
            new_dic_term_metric = {}
            
            # Nº de documentos en la especialidad que tienen el n-gram
            new_dic_term_metric['nde'] = len(list(set(list_doc_term)))
            
            #Nº de documentos en la especialidad que NO tienen el n-gram
            new_dic_term_metric['ndep'] = len(list(set(list_doc_specialty))) - len(list(set(list_doc_term)))
            
            #Número de documentos en las OTRAS especialidades (subcorpus) que tienen el n-gram
            new_dic_term_metric['ndpe'] = obtain_ndpe(term, dic_spe, specialty)
            
            # Et: nº de especialidades que contienen el ngram
            new_dic_term_metric['et'] =  obtain_et(term, dic_spe)
            
            #P(e|d)= Specialty probability of the n-gram = N(d,e)/(N(d,e) +N(d,e'))
            new_dic_term_metric['ped'] = new_dic_term_metric['nde'] / (new_dic_term_metric['nde'] + new_dic_term_metric['ndep'])
            
            #P(d|e)= Document probability of the n-gram = N(d,e)/(N(d,e) +N(d',e))
            new_dic_term_metric['pde'] = new_dic_term_metric['nde'] / (new_dic_term_metric['nde'] + new_dic_term_metric['ndpe'])
            
            new_dic_term[term] = new_dic_term_metric
            
        new_dic_metrics[specialty] = new_dic_term
        
    return new_dic_metrics

In [11]:
path_dic = 'dic_specialties.pkl'
dic_spe = deserialize_object(path_dic)

Cargado el objeto dic_specialties.pkl


In [12]:
dic_metrics = read_diccionary_specialties(dic_spe)

Especialidad: bariatric-medicine.csv
Especialidad: addiction-medicine.csv
Especialidad: genetics,-medical.csv
Especialidad: general-practice.csv
Especialidad: geography,-medical.csv
Especialidad: anesthesiology.csv
Especialidad: clinical-medicine.csv
Especialidad: geriatrics.csv
Especialidad: reproductive-medicine.csv
Especialidad: internal-medicine.csv
Especialidad: specialties,-surgical.csv
Especialidad: venereology.csv
Especialidad: regenerative-medicine.csv
Especialidad: osteopathic-medicine.csv
Especialidad: wilderness-medicine.csv
Especialidad: pathology.csv
Especialidad: integrative-medicine.csv
Especialidad: global-health.csv
Especialidad: behavioral-medicine.csv
Especialidad: adolescent-medicine.csv
Especialidad: military-medicine.csv
Especialidad: sports-medicine.csv
Especialidad: dermatology.csv
Especialidad: theranostic-nanomedicine.csv
Especialidad: physical-and-rehabilitation-medicine.csv
Especialidad: travel-medicine.csv
Especialidad: molecular-medicine.csv
Especialidad: